In [ ]:
#carregando bibliotecas
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Diretório onde estão os arquivos
pasta_arquivos = "dados" 

In [ ]:
# Arquivos dos Extrato de doações

# Listar todos os arquivos que seguem o mesmo padrão
arquivos = [f for f in os.listdir(pasta_arquivos) if f.startswith("Extrato_31_") and f.endswith(".xlsx")]

# Lista para armazenar os DataFrames
dfs = []

# Ler cada arquivo e adicioná-lo à lista
total_arquivos = len(arquivos)
print(f"Encontrados {total_arquivos} arquivos para consolidar...")
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta_arquivos, arquivo)
    df = pd.read_excel(caminho_arquivo)  # Lendo o arquivo
    df["Fonte"] = arquivo  # Adicionar coluna de origem para referência
    dfs.append(df)  # Adicionando à lista

# Concatenar todos os DataFrames
consolidado_arrecadado = pd.concat(dfs, ignore_index=True)
#transformando data em data
consolidado_arrecadado["Data"] = pd.to_datetime(consolidado_arrecadado["Data"], errors="coerce", dayfirst=True)
#criando mes ano
consolidado_arrecadado["AnoMes"] = consolidado_arrecadado["Data"].dt.to_period("M")
#criando ano
consolidado_arrecadado["Ano"] = consolidado_arrecadado["Data"].dt.year

consolidado_arrecadado

In [ ]:
# Caminho do arquivo Excel
arquivo_custos = "dados\Custos_mensal.xlsx" 

# Carregar todas as abas do arquivo
xls = pd.ExcelFile(arquivo_custos)

# Filtrar apenas as abas que seguem o padrão 'gastos_mes_ano'
abas_validas = [sheet for sheet in xls.sheet_names if sheet.startswith("gastos_")]

# Lista para armazenar os DataFrames
dfs = []

print(f"Encontradas {len(abas_validas)} abas para consolidar...")
for aba in abas_validas:
    df = pd.read_excel(xls, sheet_name=aba, usecols=[0, 1, 2, 3, 4], header=0)  # Lendo cada aba
    df["Fonte"] = aba  # Adicionar coluna de origem para referência
    dfs.append(df)

# Concatenar todas as abas
consolidado_custos = pd.concat(dfs, ignore_index=True)

#transformando data em data
consolidado_custos["Data"] = pd.to_datetime(consolidado_custos["Data"], errors="coerce", dayfirst=True)
#criando mes ano
consolidado_custos["AnoMes"] = consolidado_custos["Data"].dt.to_period("M")
#criando ano
consolidado_custos["Ano"] = consolidado_custos["Data"].dt.year

consolidado_custos

In [ ]:
# Caminho do arquivo Excel
arquivo_marmitas = "dados\Descricao_Marmitas.xlsx"  
descricao_marmitas = pd.read_excel(arquivo_marmitas)

# Remover linhas onde as colunas principais estão em branco
descricao_marmitas = descricao_marmitas.dropna(subset=["Data", "Quantidade", "Cardápio"])

#Substituindo NaN por 0 nas quantidades de alimentos
descricao_marmitas.iloc[:, 4:11] = descricao_marmitas.iloc[:, 4:11].fillna(0)

#transformando data em data
descricao_marmitas["Data"] = pd.to_datetime(descricao_marmitas["Data"], errors="coerce", dayfirst=True)
#criando mes ano
descricao_marmitas["AnoMes"] = descricao_marmitas["Data"].dt.to_period("M")
#criando ano
descricao_marmitas["Ano"] = descricao_marmitas["Data"].dt.year

descricao_marmitas

## Explorar base  consolidado_arrecadado:

Validando os dados
1. As arrecadações batem, mensalmente (isso quer dizer que tem que somar por mês) com o relatório disponível? 

Explorando um pouco
1. Qual a menor e maior data?
2. Quantos doadores distintos existem?
3. Quantos doadores distintos existem por mês? Existe algum comportamento ao longo do tempo? (gráfico de linha)
4. Qual a distribuição do valores doados de uma forma geral? (descritiva, histograma e boxplot)
5. Como é a média de valor doado por mês?  Existe algum comportamento ao longo do tempo? (gráfico de linha)
6. Quem são os doadores que mais doaram de forma geral? (somando todas as doações)
7. Quem são os doadores que mais doaram em quantidade de vezes? (quantidade de doações)
8. Existem doadores que fazem tempo que não doam? (ordenar do maior para menor)
9. Quais são as formas de pagamento e suas frequências?
10. Como está a soma de arrecadações por mês? (gráfico de linhas)

In [ ]:
#Validando os dados
#As arrecadações batem, mensalmente (isso quer dizer que tem que somar por mês) com o relatório disponível?
consolidado_arrecadado_mes_ano = consolidado_arrecadado.groupby("AnoMes")["Valor"].sum().reset_index()
consolidado_arrecadado_mes_ano

In [ ]:
#1. Qual a menor e maior data?
menor_data = consolidado_arrecadado["Data"].min()
maior_data = consolidado_arrecadado["Data"].max()

print(f"Menor data: {menor_data}, Maior data: {maior_data}")

In [ ]:
#2. Quantos doadores distintos existem?
doadores_distintos = consolidado_arrecadado["ID"].nunique()
print(f"Total de doadores distintos: {doadores_distintos}")

In [ ]:
#3. Quantos doadores distintos existem por mês? Existe algum comportamento ao longo do tempo? (gráfico de linha)
doadores_por_mes = consolidado_arrecadado.groupby("AnoMes")["ID"].nunique().reset_index()

plt.figure(figsize=(10, 5))
plt.plot(doadores_por_mes["AnoMes"].astype(str), doadores_por_mes["ID"], marker='o', linestyle='-')
plt.xticks(rotation=90)
plt.xlabel("Ano-Mês")
plt.ylabel("Doadores distintos")
plt.title("Quantidade de doadores distintos ao longo do tempo")
plt.grid(True)
plt.show()


In [ ]:
#4. Qual a distribuição do valores doados de uma forma geral? (descritiva, histograma e boxplot)

# Estatísticas descritivas
print(consolidado_arrecadado["Valor"].describe())

# Criando a figura e os eixos compartilhados
fig, ax = plt.subplots(2, 1, figsize=(10, 8), gridspec_kw={"height_ratios": [3, 1]}, sharex=True)

# Histograma
sns.histplot(consolidado_arrecadado["Valor"], bins=30, kde=True, ax=ax[0])
ax[0].set_ylabel("Frequência")
ax[0].set_title("Distribuição dos Valores Doados")

# Boxplot com pontos individuais (stripplot)
sns.boxplot(x=consolidado_arrecadado["Valor"], ax=ax[1], showfliers=False)
sns.stripplot(x=consolidado_arrecadado["Valor"], ax=ax[1], color="black", alpha=0.3, jitter=True, size=3)
ax[1].set_xlabel("Valor da Doação")
ax[1].set_ylabel("")

# Ajuste de layout
plt.tight_layout()
plt.show()



In [ ]:
#5. Como é a média de valor doado por mês?  Existe algum comportamento ao longo do tempo? (gráfico de linha)
media_valor_mes = consolidado_arrecadado.groupby("AnoMes")["Valor"].mean().reset_index()

plt.figure(figsize=(10, 5))
plt.plot(media_valor_mes["AnoMes"].astype(str), media_valor_mes["Valor"], marker='o', linestyle='-')
plt.xticks(rotation=90)
plt.xlabel("Ano-Mês")
plt.ylabel("Média do Valor Doadado")
plt.title("Evolução da Média de Doações ao Longo do Tempo")
plt.grid(True)
plt.show()


In [ ]:
#6. Quem são os doadores que mais doaram de forma geral? (somando todas as doações)
top_doadores = consolidado_arrecadado.groupby("ID")["Valor"].sum().reset_index()
top_doadores = top_doadores.sort_values(by="Valor", ascending=False)
top_doadores

In [ ]:
#7. Quem são os doadores que mais doaram em quantidade de vezes? (quantidade de doações)
top_doadores = consolidado_arrecadado.groupby("ID").agg(
    Quantidade_Doacoes=("Valor", "count"),
    Total_Valor_Doado=("Valor", "sum")
).reset_index()

# Ordenar pelos maiores valores doados
top_doadores = top_doadores.sort_values(by="Total_Valor_Doado", ascending=False)
top_doadores.head(40)

In [ ]:
#8. Existem doadores que fazem tempo que não doam? (ordenar do maior para menor)
# Calcular a última doação de cada doador
ultima_doacao = consolidado_arrecadado.groupby("ID")["Data"].max().reset_index()

# Calcular a diferença em meses da última doação para hoje
hoje = pd.to_datetime("today")
ultima_doacao["Meses_Desde_Ultima"] = (hoje.year - ultima_doacao["Data"].dt.year) * 12 + (hoje.month - ultima_doacao["Data"].dt.month)

# Ordenar pelos doadores que estão há mais tempo sem doar
ultima_doacao = ultima_doacao.sort_values(by="Meses_Desde_Ultima", ascending=False)
ultima_doacao

In [ ]:
#9. Quais são as formas de pagamento e suas frequências?
formas_pagamento = consolidado_arrecadado["Forma_Pagamento"].value_counts().reset_index()
formas_pagamento.columns = ["Forma_Pagamento", "Frequência"]
formas_pagamento

In [ ]:
#10. Como está a soma de arrecadações por mês? (gráfico de linhas)
consolidado_arrecadado_mes_ano
plt.figure(figsize=(10, 5))
plt.plot(consolidado_arrecadado_mes_ano["AnoMes"].astype(str), consolidado_arrecadado_mes_ano["Valor"], marker='o', linestyle='-')
plt.xticks(rotation=90)
plt.xlabel("Ano-Mês")
plt.ylabel("Total Arrecadado")
plt.title("Evolução da Arrecadação Mensal")
plt.grid(True)
plt.show()


In [ ]:
#para observar melhor, colocando os 3 gráficos juntos

# Definindo as datas onde queremos as linhas verticais
linhas_verticais = ["2021-06", "2021-09", "2023-01", "2024-02"]

fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# 1. Evolução da Arrecadação Mensal
axes[0].plot(consolidado_arrecadado_mes_ano["AnoMes"].astype(str), 
             consolidado_arrecadado_mes_ano["Valor"], 
             marker='o', linestyle='-')
axes[0].set_ylabel("Total Arrecadado")
axes[0].set_title("Evolução da Arrecadação Mensal")
axes[0].grid(True)

# 2. Evolução da Média de Doações ao Longo do Tempo
axes[1].plot(media_valor_mes["AnoMes"].astype(str), 
             media_valor_mes["Valor"], 
             marker='o', linestyle='-')
axes[1].set_ylabel("Média do Valor Doado")
axes[1].set_title("Evolução da Média de Doações ao Longo do Tempo")  # CORRIGIDO
axes[1].grid(True)

# 3. Quantidade de Doadores Distintos ao Longo do Tempo
axes[2].plot(doadores_por_mes["AnoMes"].astype(str), 
             doadores_por_mes["ID"], 
             marker='o', linestyle='-')
axes[2].set_ylabel("Doadores Distintos")
axes[2].set_title("Quantidade de Doadores Distintos ao Longo do Tempo")
axes[2].set_xlabel("Ano-Mês")
axes[2].grid(True)

# Adicionando linhas verticais nos três gráficos
for ax in axes:
    for linha in linhas_verticais:
        ax.axvline(x=linha, color='r', linestyle='--', alpha=0.7)
    ax.legend()

plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
#qual o valor arrecadado no total?
total_arrecadado = consolidado_arrecadado["Valor"].sum()
print(f"Total arrecadado: {total_arrecadado}")

## Explorar base  consolidado_gastos

Validando os dados
1. Os gastos batem, mensalmente (isso quer dizer que tem que somar por mês) com o relatório disponível? 

Explorando um pouco
1. Qual a menor e maior data?
2. Quantos tipos de gastos distintos existem e suas qtdes e soma de valores?
3. Quantos locais de gastos distintos existem e suas qtdes e soma de valores?
4. Qual a distribuição do valores doados de uma forma geral? (descritiva, histograma e boxplot)
5. Como é a soma de valor gasto por mês?  Existe algum comportamento ao longo do tempo? (gráfico de linha)